In [6]:
import requests 
from bs4 import BeautifulSoup
from datetime import datetime 
from datetime import date
from datetime import timedelta
import time 
from glob import glob
import pandas as pd
import os 
from pathlib import Path
from ast import literal_eval
import re
from unidecode import unidecode
import plotly.express as px
import matplotlib.pyplot as plt

In [2]:
fundos = {
    'IP': {
        'MASCARA': '12.082.452/0001-49',
        'ROSTO' : '11.435.298/0001-89',
        'PK_PARTIC': '100377'
    },

    'DYNAMO': {
        'MASCARA': '73.232.530/0001-39',
        'ROSTO' : '37.916.879/0001-26',
        'PK_PARTIC': '190109'
    },

    'VELT': {
        'MASCARA': '',
        'ROSTO' : '08.927.452/0001-25',
        'PK_PARTIC': '73929'
    },

    'REAL INVESTOR': {
        'MASCARA': '',
        'ROSTO' : '36.352.539/0001-57',
        'PK_PARTIC': '193679'
    },

    'ATMOS': {
        'MASCARA': '',
        'ROSTO' : '11.188.572/0001-62',
        'PK_PARTIC': '94978'
    },

    'CONSTELLATION': {
        'MASCARA': '',
        'ROSTO' : '11.225.860/0001-40',
        'PK_PARTIC': '95894'
    },

   'SQUADRA': {
        'MASCARA': '09.285.146/0001-03',
        'ROSTO' : '09.412.648/0001-40',
        'PK_PARTIC': '80320'
    },

   'TEMPO': {
        'MASCARA': '00.906.044/0001-85',
        'ROSTO' : '11.046.362/0001-30',
        'PK_PARTIC': '93817'
    },

   'GUEPARDO': {
        'MASCARA': '08.830.947/0001-31',
        'ROSTO' : '14.213.077/0001-54',
        'PK_PARTIC': '122677'
    },

   'BOGARI': {
        'MASCARA': '08.323.402/0001-39',
        'ROSTO' : '15.165.493/0001-97',
        'PK_PARTIC': '125916'
    },

   'ALASKA POLAND': {
        'MASCARA': '',
        'ROSTO' : '05.775.774/0001-08',
        'PK_PARTIC': '54232'
    },

   'GERACAO L PAR': {
        'MASCARA': '',
        'ROSTO' : '08.935.128/0001-59',
        'PK_PARTIC': '73249'
    },

    '3G RADAR': {
        'MASCARA': '',
        'ROSTO' : '18.324.976/0001-85',
        'PK_PARTIC': '131418'
    },
}

P = [pk['PK_PARTIC'] for pk in list(fundos.values())]
meses_anteriores = 12
funds = []


# Percorrer os fundos 
def raspagemCarteira(PK, meses_anteriores, fundo_destino): 
    for id in PK:
        url = 'https://cvmweb.cvm.gov.br/SWB/Sistemas/SCW/CPublica/CDA/CPublicaCDA.aspx?PK_PARTIC='+id+'&SemFrame='
        http = requests.get(url)
        soup = BeautifulSoup(http.content, "html.parser" )
        nomeFundo = soup.select('#lbNmDenomSocial')[0].text
        fund = {'Fundo':nomeFundo, 'posicoes': []}
        viewState = soup.select('#__VIEWSTATE')[0].get('value')
        eventValidation = soup.select('#__EVENTVALIDATION')[0].get('value')
        

        # Percorrendo todos os meses
        for i in range(2, meses_anteriores):
            tickers = []
            ddCompt = soup.select('option')[i].get('value')
            mes = soup.select('option')[i].text
            fund['Fundo'] = nomeFundo

            params = {
            '__EVENTTARGET': 'ddCOMPTC',
            '__EVENTARGUMENT':'', 
            '__LASTFOCUS':'',
            '__VIEWSTATE':viewState,
            '__VIEWSTATEGENERATOR': 
            '1D0C00E6',
            '__EVENTVALIDATION': eventValidation, 
            'ddCOMPTC': ddCompt
            }

            x = requests.post(url, data = params)
            sp = BeautifulSoup(x.content, "html.parser" )
            
            # Percorrendo a carteira
            for index in sp.find_all('table')[1].find_all('tr', attrs = {'bgcolor': ['#FAEFCA', 'Cornsilk']}):
                line = index.find_all('td', attrs = {'align': 'left'})[0].text

                if unidecode(line.split(':')[0].split('.')[0].strip().lower()) == 'acoesdescricao' or unidecode(line.split(':')[0].split('.')[0].strip().lower()) =='acoescod' or unidecode(line.split(':')[0].split('.')[0].strip().lower()) == 'athSSescod':
                    word = line.split(':')[1].strip().split(' ')[0]
                    tick = re.search('^.*\d', word).group(0)
                    pl = index.find_all('td')[-1].text
                    qtd = index.find_all('td', attrs = {'align': 'center'})[6].text
                
                    tickers.append({'Ticker': tick, '% Pl': pl, 'Quantidade': qtd})
            
            dic = {
                'Mes': mes,
                'Posicoes': tickers
            }

            fund['posicoes'].append(dic)
        fundo_destino.append(fund)

raspagemCarteira(P, meses_anteriores, funds)

df = pd.DataFrame(funds)
df.to_csv('MVP-Pront.csv', index=False)


KeyboardInterrupt



In [8]:
df = pd.read_csv('./MVP-Pront.csv')
array_funds = df['Fundo'].tolist()

In [11]:
df['posicoes'][0]

"[{'Mes': '12/2022', 'Posicoes': [{'Ticker': 'ITSA4', '% Pl': '6,761', 'Quantidade': '4.202.271'}, {'Ticker': 'EQTL3', '% Pl': '4,473', 'Quantidade': '875.650'}, {'Ticker': 'RDOR3', '% Pl': '2,092', 'Quantidade': '374.144'}, {'Ticker': 'HAPV3', '% Pl': '1,429', 'Quantidade': '1.488.025'}, {'Ticker': 'MULT3', '% Pl': '0,574', 'Quantidade': '138.679'}]}, {'Mes': '11/2022', 'Posicoes': [{'Ticker': 'ITSA4', '% Pl': '5,488', 'Quantidade': '3.400.253'}, {'Ticker': 'HAPV3', '% Pl': '4,214', 'Quantidade': '4.458.900'}, {'Ticker': 'EQTL3', '% Pl': '3,79', 'Quantidade': '756.945'}, {'Ticker': 'RDOR3', '% Pl': '1,474', 'Quantidade': '261.075'}, {'Ticker': 'MULT3', '% Pl': '0,71', 'Quantidade': '170.372'}]}, {'Mes': '10/2022', 'Posicoes': [{'Ticker': 'HAPV3', '% Pl': '6,087', 'Quantidade': '4.368.563'}, {'Ticker': 'EQTL3', '% Pl': '5,76', 'Quantidade': '1.073.713'}, {'Ticker': 'ITSA4', '% Pl': '4,6', 'Quantidade': '2.397.658'}, {'Ticker': 'MULT3', '% Pl': '2,57', 'Quantidade': '545.749'}, {'Ticker

In [3]:
def get_initial_final_position(datafram):
    for i in range(len(datafram['posicoes'])):
        posInicial = eval(datafram['posicoes'][i])[-1]
        posFinal = eval(datafram['posicoes'][i])[0]
        datafram.loc[i, 'init_final_position'] = str([posInicial, posFinal])
    return datafram

get_initial_final_position(df)

def get_notices(x):
    notices = []
    for a in eval(x)[0]['Posicoes']:
        for b in eval(x)[-1]['Posicoes']:
            if a['Ticker'] == b['Ticker']:
                if int(a['Quantidade'].replace(".", "")) > int(b['Quantidade'].replace(".", "")):
                    notices.append(" comprou " + str(int(a['Quantidade'].replace(".", "")) - int(b['Quantidade'].replace(".", ""))) + " quantidades de " + a['Ticker'])
                else:
                    notices.append(" vendeu " + str(int(b['Quantidade'].replace(".", "")) - int(a['Quantidade'].replace(".", ""))) + " quantidades de " + a['Ticker'])
    return notices

def get_final_notice(datafram):
    df['news'] = df['init_final_position'].apply(get_notices)
    absolute_notice = []
    for i in datafram['Fundo']:
        nomeFundo = ' '.join(i.split()[:2])
        array_newFinal = []
        for a in datafram['news'].loc[datafram['Fundo'] == i]:
            for b in a:
                new_final = nomeFundo + b
                array_newFinal.append(new_final)
                absolute_notice.append(new_final)
        datafram.loc[datafram['Fundo'] == i, 'news_final'] = str(array_newFinal)
    return absolute_notice
            




In [5]:
# criar uma função que vincula o nome do fundo com a movimentação
# criar um for que percorre todos os elementos do news para cada fundo e adiciona o nome do fundo no começo 

# w = get_final_notice(df)
# w

df['posicoes'][0]

"[{'Mes': '12/2022', 'Posicoes': [{'Ticker': 'ITSA4', '% Pl': '6,761', 'Quantidade': '4.202.271'}, {'Ticker': 'EQTL3', '% Pl': '4,473', 'Quantidade': '875.650'}, {'Ticker': 'RDOR3', '% Pl': '2,092', 'Quantidade': '374.144'}, {'Ticker': 'HAPV3', '% Pl': '1,429', 'Quantidade': '1.488.025'}, {'Ticker': 'MULT3', '% Pl': '0,574', 'Quantidade': '138.679'}]}, {'Mes': '11/2022', 'Posicoes': [{'Ticker': 'ITSA4', '% Pl': '5,488', 'Quantidade': '3.400.253'}, {'Ticker': 'HAPV3', '% Pl': '4,214', 'Quantidade': '4.458.900'}, {'Ticker': 'EQTL3', '% Pl': '3,79', 'Quantidade': '756.945'}, {'Ticker': 'RDOR3', '% Pl': '1,474', 'Quantidade': '261.075'}, {'Ticker': 'MULT3', '% Pl': '0,71', 'Quantidade': '170.372'}]}, {'Mes': '10/2022', 'Posicoes': [{'Ticker': 'HAPV3', '% Pl': '6,087', 'Quantidade': '4.368.563'}, {'Ticker': 'EQTL3', '% Pl': '5,76', 'Quantidade': '1.073.713'}, {'Ticker': 'ITSA4', '% Pl': '4,6', 'Quantidade': '2.397.658'}, {'Ticker': 'MULT3', '% Pl': '2,57', 'Quantidade': '545.749'}, {'Ticker

In [98]:
for i in df['Fundo']:
    nomeFundo = ' '.join(i.split()[:2])
    array_newFinal = []
    for a in df['news'].loc[df['Fundo'] == i]:
        for b in a:
            new_final = nomeFundo + b
            array_newFinal.append(new_final)
    df.loc[df['Fundo'] == i, 'news_final'] = str(array_newFinal)
            



In [101]:
df['news_final'][0]

"['IP PARTICIPAÇŐES comprou 1122431 quantidades de EQTL3', 'IP PARTICIPAÇŐES comprou 2544102 quantidades de HAPV3', 'IP PARTICIPAÇŐES comprou 260424 quantidades de MULT3', 'IP PARTICIPAÇŐES vendeu 209050 quantidades de RDOR3']"

# Validando o código PK dos fundos

In [90]:
P = [pk['PK_PARTIC'] for pk in list(fundos.values())]
meses_anteriores = 12

In [91]:
for cod in P:
    print(cod)
    url = 'https://cvmweb.cvm.gov.br/SWB/Sistemas/SCW/CPublica/CDA/CPublicaCDA.aspx?PK_PARTIC='+cod+'&SemFrame='
    http = requests.get(url)
    soup = BeautifulSoup(http.content, "html.parser" )
    print(soup.select('#lbNmDenomSocial'))
    break

100377
[<span id="lbNmDenomSocial">IP PARTICIPAÇŐES MASTER FUNDO DE INVESTIMENTO EM AÇŐES BDR NÍVEL I</span>]


# Coletando a carteira de um único fundo 

In [92]:
# Coletando o viewstate para fazer o POST das carteiras

viewState = soup.select('#__VIEWSTATE')[0].get('value')
eventValidation = soup.select('#__EVENTVALIDATION')[0].get('value')
ddCompt = soup.select('option')[4].get('value')
params ={
'__EVENTTARGET': 'ddCOMPTC',
'__EVENTARGUMENT':'', 
'__LASTFOCUS':'',
'__VIEWSTATE':viewState,
'__VIEWSTATEGENERATOR': 
'1D0C00E6',
'__EVENTVALIDATION': eventValidation, 
'ddCOMPTC': ddCompt
}
x = requests.post(url, data = params)
sp = BeautifulSoup(x.content, "html.parser" )


# Tentando pegar a ultima coluna (%pl)

In [107]:
# coletando o left do ativo, que informa se é ação ou não e o pl
for i in sp.find_all('table')[1].find_all('tr', attrs = {'bgcolor': ['#FAEFCA', 'Cornsilk']}):
    # a = i.find_all('td')[-1].text
    # b = i.find_all('td', attrs = {'align': 'left'})[0].text
    # c = i.find_all('td', attrs = {'align': 'center'})[6].text

    

    # if unidecode(b.split(':')[0].split('.')[0].strip().lower()) == 'acoesdescricao' or unidecode(b.split(':')[0].split('.')[0].strip().lower()) =='acoescod' or unidecode(b.split(':')[0].split('.')[0].strip().lower()) == 'athSSescod':
    #     word = b.split(':')[1].strip().split(' ')[0]
    #     tick = re.search('^.*\d', word).group(0)
    #     print('Comeu ' + tick + ' e o PL dele é ' + a)

    break


[<td align="center"><span id="dlAplics__ctl1_lbNConfTpNegoc">Para<br/>negociaçăo</span></td>, <td align="center"><span id="dlAplics__ctl1_lbNConfEmpLigada">Năo</span></td>, <td align="center">0</td>, <td align="center"><span id="dlAplics__ctl1_lbNConfVlVendasNegoc">0,00</span></td>, <td align="center">0</td>, <td align="center"><span id="dlAplics__ctl1_lbNConfVlAquisNegoc">0,00</span></td>, <td align="center">4.368.563</td>, <td align="center"><span id="dlAplics__ctl1_lbNConfVlCustoCorrecPosFim"></span></td>, <td align="center"><span id="dlAplics__ctl1_lbNConfVlPosFim">34.074.791,40</span></td>, <td align="center">6,087</td>]


# Empacotando o códido e rodando todos os fundos

In [32]:
# inputs: meses anteriores, lista de fundos, 

funds = []

# Percorrer os fundos 
for id in PK:
    url = 'https://cvmweb.cvm.gov.br/SWB/Sistemas/SCW/CPublica/CDA/CPublicaCDA.aspx?PK_PARTIC='+id+'&SemFrame='
    http = requests.get(url)
    soup = BeautifulSoup(http.content, "html.parser" )
    nomeFundo = soup.select('#lbNmDenomSocial')[0].text
    fund = {'Fundo':nomeFundo, 'posicoes': []}
    viewState = soup.select('#__VIEWSTATE')[0].get('value')
    eventValidation = soup.select('#__EVENTVALIDATION')[0].get('value')
    

    # Percorrendo todos os meses
    for i in range(2, meses_anteriores):
        tickers = []
        ddCompt = soup.select('option')[i].get('value')
        mes = soup.select('option')[i].text
        fund['Fundo'] = nomeFundo

        params = {
        '__EVENTTARGET': 'ddCOMPTC',
        '__EVENTARGUMENT':'', 
        '__LASTFOCUS':'',
        '__VIEWSTATE':viewState,
        '__VIEWSTATEGENERATOR': 
        '1D0C00E6',
        '__EVENTVALIDATION': eventValidation, 
        'ddCOMPTC': ddCompt
        }

        x = requests.post(url, data = params)
        sp = BeautifulSoup(x.content, "html.parser" )
        
        # Percorrendo a carteira
        for index in range(len(sp.select("[align=left]"))):
            line = sp.select("[align=left]")[index].text

            if unidecode(line.split(':')[0].split('.')[0].strip().lower()) == 'acoesdescricao' or unidecode(line.split(':')[0].split('.')[0].strip().lower()) =='acoescod' or unidecode(line.split(':')[0].split('.')[0].strip().lower()) == 'athSSescod':
            # CRIAR CONDIÇÃO QUE SÓ PEGA O TICKER SE FOR AÇÃO, PRECISO ENDIREITAR A STRING DO TICKER QUE FICA COLADA COM A PALAVRA DE BAIXO
            # POSSO USAR REGEX PRA PEGAR APENAS A PALAVRA ATÉ O NUMERO
                word = line.split(':')[1].strip().split(' ')[0]
                tick = re.search('^.*\d', word).group(0)

                tickers.append(tick)
        
        dic = {
            'Mes': mes,
            'Posicoes': tickers
        }

        fund['posicoes'].append(dic)
    funds.append(fund)

ConnectionError: HTTPSConnectionPool(host='cvmweb.cvm.gov.br', port=443): Max retries exceeded with url: /SWB/Sistemas/SCW/CPublica/CDA/CPublicaCDA.aspx?PK_PARTIC=73249&SemFrame= (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x00000243BA752DC0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

# Função modulada

In [3]:
funds = []

# Percorrer os fundos 
def raspagemCarteira(PK, meses_anteriores, fundo_destino): 
    for id in PK:
        url = 'https://cvmweb.cvm.gov.br/SWB/Sistemas/SCW/CPublica/CDA/CPublicaCDA.aspx?PK_PARTIC='+id+'&SemFrame='
        http = requests.get(url)
        soup = BeautifulSoup(http.content, "html.parser" )
        nomeFundo = soup.select('#lbNmDenomSocial')[0].text
        fund = {'Fundo':nomeFundo, 'posicoes': []}
        viewState = soup.select('#__VIEWSTATE')[0].get('value')
        eventValidation = soup.select('#__EVENTVALIDATION')[0].get('value')
        

        # Percorrendo todos os meses
        for i in range(2, meses_anteriores):
            tickers = []
            ddCompt = soup.select('option')[i].get('value')
            mes = soup.select('option')[i].text
            fund['Fundo'] = nomeFundo

            params = {
            '__EVENTTARGET': 'ddCOMPTC',
            '__EVENTARGUMENT':'', 
            '__LASTFOCUS':'',
            '__VIEWSTATE':viewState,
            '__VIEWSTATEGENERATOR': 
            '1D0C00E6',
            '__EVENTVALIDATION': eventValidation, 
            'ddCOMPTC': ddCompt
            }

            x = requests.post(url, data = params)
            sp = BeautifulSoup(x.content, "html.parser" )
            
            # Percorrendo a carteira
            for index in range(len(sp.select("[align=left]"))):
                line = sp.select("[align=left]")[index].text

                if unidecode(line.split(':')[0].split('.')[0].strip().lower()) == 'acoesdescricao' or unidecode(line.split(':')[0].split('.')[0].strip().lower()) =='acoescod' or unidecode(line.split(':')[0].split('.')[0].strip().lower()) == 'athSSescod':
                # CRIAR CONDIÇÃO QUE SÓ PEGA O TICKER SE FOR AÇÃO, PRECISO ENDIREITAR A STRING DO TICKER QUE FICA COLADA COM A PALAVRA DE BAIXO
                # POSSO USAR REGEX PRA PEGAR APENAS A PALAVRA ATÉ O NUMERO
                    word = line.split(':')[1].strip().split(' ')[0]
                    tick = re.search('^.*\d', word).group(0)

                    tickers.append(tick)
            
            dic = {
                'Mes': mes,
                'Posicoes': tickers
            }

            fund['posicoes'].append(dic)
        funds.append(fund)
    return funds